# Indie Vs Triple A

The gaming industry has seen a shift in recent years, that being the rise of independent developers. In the early 2000s, the ide

In [1]:
# Import Steam Dataset

import pandas as pd

dataset = pd.read_json("games.json")
dataset = dataset.transpose()

In [2]:
# Filtering

dataset = dataset[dataset['reviews'].str.len() > 0]

In [4]:
dataset.sample(5)

,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,header_image,...,score_rank,positive,negative,estimated_owners,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags
1031790,Eternower,"Mar 29, 2020",0,1.0,0,Game Eternower is a mazing-focused Tower Defen...,Game Eternower is a mazing-focused Tower Defen...,Eternower is a mazing-focused Tower Defense. M...,"“looks kinda neat, i guess” &gt;&gt;281935424",https://cdn.akamai.steamstatic.com/steam/apps/...,...,,45,4,0 - 20000,0,0,0,0,0,"{'Tower Defense': 123, 'Minimalist': 117, '2D'..."
651490,No Longer Home,"Jul 30, 2021",0,14.99,2,Bo and Ao are graduating university and prepar...,Bo and Ao are graduating university and prepar...,No Longer Home is a game about letting go of t...,"“[No Longer Home, formally known as] 29 is a r...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,,155,43,0 - 20000,0,0,0,0,2,"{'Interactive Fiction': 329, 'LGBTQ+': 315, 'S..."
339200,Oceanhorn: Monster of Uncharted Seas,"Mar 17, 2015",0,14.99,0,You wake up and find a letter from your father...,You wake up and find a letter from your father...,Oceanhorn: Monster of Uncharted Seas combines ...,“Does a great job of reproducing the magic spa...,https://cdn.akamai.steamstatic.com/steam/apps/...,...,,1485,374,200000 - 500000,176,0,197,0,4,"{'Adventure': 148, 'RPG': 134, 'Action': 124, ..."
691650,Kaet Must Die!,"Apr 5, 2018",0,14.99,0,"This is the hardest, most challenging, horror ...","This is the hardest, most challenging, horror ...","This is the hardest, most challenging, horror ...",“Any sucker for punishment needs to check out ...,https://cdn.akamai.steamstatic.com/steam/apps/...,...,,9,3,0 - 20000,0,0,0,0,0,"{'Indie': 32, 'Gore': 31, 'Violent': 31, 'Acti..."
1788520,OctoMaze,"May 9, 2022",0,3.99,0,OctoMaze is a breath of fresh air in the puzzl...,OctoMaze is a breath of fresh air in the puzzl...,Priceless artifacts were stolen long ago from ...,“LaunchPad #5” PocketGamer “Sydney Butler” Onl...,https://cdn.akamai.steamstatic.com/steam/apps/...,...,,1,0,0 - 20000,0,0,0,0,0,"{'Casual': 62, 'Puzzle': 36, '2D': 33, 'Logic'..."
